In [1]:
import tensorflow as tf
# #指定使用那块GUP训练
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf.ConfigProto()
# 设置最大占有GPU不超过显存的70%
config.gpu_options.per_process_gpu_memory_fraction = 0.7 
# # 重点：设置动态分配GPU
config.gpu_options.allow_growth = True
# 创建session时
tf.Session(config=config) 

In [3]:
import numpy as np
from keras.datasets import cifar10
from torch.utils.data import Dataset
import torch.utils.data as Data
class MyDataset(Dataset):
    def __init__(self, imgs, transform=None):
        # super().__init__()
        self.imgs = imgs
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img)
        img=img.reshape([3,32,32])
        return img
import torch
from torch.autograd import Variable
import model
import torch.nn.functional as F
model = model.cifar10(128)
model.load_state_dict(torch.load('./log/default/best-85.pth'))
model.cuda()
def EuclideanDistances(A, B):
    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(), ED.shape[0] * ED.shape[1])


def cal_distance_image_real(images, labels):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=200, shuffle=True)
    y_logits = []
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]
    dict = {}
    all_dis = []
    for i in range(10):
        dict[i] = []
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i] = np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)


def cal_distance_image_fake(images):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=200, shuffle=True)
    y_logits = []
    labels=[]
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = output.data.max(1)[1]
        labels += [i for i in pred.cpu().numpy()]
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]

    dict = {}
    all_dis = []
    for i in range(10):
        dict[i] = []
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i] = np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

import os
if not os.path.isdir('saved_models_{}'.format('wgan')):
    os.mkdir('saved_models_{}'.format('wgan'))
f = open('saved_models_{}/log_collapse1.txt'.format('wgan'), mode='w')
import torch.utils.data as Data
import cv2


from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import RMSprop

import keras.backend as K
from PIL import Image
import matplotlib.pyplot as plt
import os
import sys

import numpy as np


class WGAN():
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        self.clip_value = 0.01
        optimizer = RMSprop(lr=0.00005)

        # Build and compile the critic
        self.critic = self.build_critic()
        self.critic.compile(loss=self.wasserstein_loss,
                            optimizer=optimizer,
                            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.critic.trainable = False

        # The critic takes generated images as input and determines validity
        valid = self.critic(img)

        # The combined model  (stacked generator and critic)
        self.combined = Model(z, valid)
        self.combined.compile(loss=self.wasserstein_loss,
                              optimizer=optimizer,
                              metrics=['accuracy'])

    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=4, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_critic(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        # model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (X_test, y_test) = cifar10.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = -np.ones((batch_size, 1))
        fake = np.ones((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0
        steps = []
        values = []

        for epoch in range(epochs):

            for index in range(nb_batches):
                for _ in range(self.n_critic):
                    global_step += 1
                    imgs = X_train[index * batch_size:(index + 1) * batch_size]
                    # ---------------------
                    #  Train Discriminator
                    # ---------------------

                    # Select a random batch of images
                    # idx = np.random.randint(0, X_train.shape[0], batch_size)
                    # imgs = X_train[idx]

                    # Sample noise as generator input
                    noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                    # Generate a batch of new images
                    gen_imgs = self.generator.predict(noise)

                    # Train the critic
                    d_loss_real = self.critic.train_on_batch(imgs, valid)
                    d_loss_fake = self.critic.train_on_batch(gen_imgs, fake)
                    d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)

                    # Clip critic weights
                    for l in self.critic.layers:
                        weights = l.get_weights()
                        weights = [np.clip(w, -self.clip_value, self.clip_value) for w in weights]
                        l.set_weights(weights)

                    # ---------------------
                    #  Train Generator
                    # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d step:%d[D loss: %f] [G loss: %f]" % (epoch, global_step, 1 - d_loss[0], 1 - g_loss[0]))
                sample_num=5000

                if global_step % sample_interval == 0:
                    self.mode_drop(X_test,y_test,sample_num, global_step)


    def mode_drop(self, x_test,y_test,sample_num, global_step):
        r, c = 10, 1000
        noise = np.random.normal(0, 1, (r * c, 100))
        # sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('step:' + str(global_step))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')

if __name__ == '__main__':
    wgan = WGAN()
    wgan.train(epochs=20, batch_size=64, sample_interval=200)


Sequential(
  (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (5): ReLU()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (9): ReLU()
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (12): ReLU()
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): BatchNorm2d(512, eps=1e-05

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 16)        448       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 32)          4640      
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 32)          128       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 32)          0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:5[D loss: 0.999906] [G loss: 1.000187]
epoch:0 step:10[D loss: 0.999914] [G loss: 1.000194]
epoch:0 step:15[D loss: 0.999915] [G loss: 1.000194]
epoch:0 step:20[D loss: 0.999912] [G loss: 1.000190]
epoch:0 step:25[D loss: 0.999912] [G loss: 1.000190]
epoch:0 step:30[D loss: 0.999914] [G loss: 1.000188]
epoch:0 step:35[D loss: 0.999917] [G loss: 1.000182]
epoch:0 step:40[D loss: 0.999917] [G loss: 1.000189]
epoch:0 step:45[D loss: 0.999918] [G loss: 1.000188]
epoch:0 step:50[D loss: 0.999913] [G loss: 1.000187]
epoch:0 step:55[D loss: 0.999918] [G loss: 1.000186]
epoch:0 step:60[D loss: 0.999921] [G loss: 1.000182]
epoch:0 step:65[D loss: 0.999922] [G loss: 1.000175]
epoch:0 step:70[D loss: 0.999922] [G loss: 1.000172]
epoch:0 step:75[D loss: 0.999921] [G loss: 1.000163]
epoch:0 step:80[D loss: 0.999924] [G loss: 1.000157]
epoch:0 step:85[D loss: 0.999930] [G loss: 1.000140]
epoch:0 step:90[D loss: 0.999931] [G loss: 1.000141]
epoch:0 step:95[D loss: 0.999934] [G loss: 1.00

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:57: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:59: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:86: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:90: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


...

epoch:19 step:75105[D loss: 0.999992] [G loss: 1.000030]
epoch:19 step:75110[D loss: 1.000075] [G loss: 0.999968]
epoch:19 step:75115[D loss: 0.999947] [G loss: 1.000044]
epoch:19 step:75120[D loss: 1.000052] [G loss: 0.999992]
epoch:19 step:75125[D loss: 0.999906] [G loss: 1.000060]
epoch:19 step:75130[D loss: 0.999967] [G loss: 1.000051]
epoch:19 step:75135[D loss: 0.999972] [G loss: 1.000041]
epoch:19 step:75140[D loss: 0.999997] [G loss: 1.000028]
epoch:19 step:75145[D loss: 1.000020] [G loss: 0.999987]
epoch:19 step:75150[D loss: 0.999951] [G loss: 1.000010]
epoch:19 step:75155[D loss: 0.999974] [G loss: 1.000090]
epoch:19 step:75160[D loss: 0.999970] [G loss: 1.000070]
epoch:19 step:75165[D loss: 0.999982] [G loss: 1.000033]
epoch:19 step:75170[D loss: 0.999957] [G loss: 1.000085]
epoch:19 step:75175[D loss: 0.999975] [G loss: 1.000068]
epoch:19 step:75180[D loss: 0.999990] [G loss: 1.000041]
epoch:19 step:75185[D loss: 1.000038] [G loss: 1.000030]
epoch:19 step:75190[D loss: 0.9

epoch:19 step:75790[D loss: 1.000024] [G loss: 0.999882]
epoch:19 step:75795[D loss: 0.999980] [G loss: 1.000391]
epoch:19 step:75800[D loss: 0.999943] [G loss: 1.000173]
##############
[0.86549237 0.86472786 0.8317422  0.81786713 0.79239183 0.85694764
 0.87103404 0.81951732 0.83220937 0.82141114]
##########
epoch:19 step:75805[D loss: 1.000014] [G loss: 0.999875]
epoch:19 step:75810[D loss: 0.999726] [G loss: 1.000543]
epoch:19 step:75815[D loss: 0.999923] [G loss: 1.000073]
epoch:19 step:75820[D loss: 0.999995] [G loss: 1.000007]
epoch:19 step:75825[D loss: 1.000028] [G loss: 0.999949]
epoch:19 step:75830[D loss: 0.999936] [G loss: 1.000059]
epoch:19 step:75835[D loss: 1.000079] [G loss: 0.999916]
epoch:19 step:75840[D loss: 0.999802] [G loss: 1.000185]
epoch:19 step:75845[D loss: 1.000183] [G loss: 0.999865]
epoch:19 step:75850[D loss: 1.000082] [G loss: 1.000023]
epoch:19 step:75855[D loss: 1.000074] [G loss: 0.999905]
epoch:19 step:75860[D loss: 0.999964] [G loss: 1.000141]
epoch:

epoch:19 step:76460[D loss: 1.000071] [G loss: 0.999885]
epoch:19 step:76465[D loss: 0.999999] [G loss: 0.999952]
epoch:19 step:76470[D loss: 1.000095] [G loss: 0.999883]
epoch:19 step:76475[D loss: 0.999962] [G loss: 1.000104]
epoch:19 step:76480[D loss: 0.999989] [G loss: 1.000080]
epoch:19 step:76485[D loss: 0.999947] [G loss: 1.000200]
epoch:19 step:76490[D loss: 0.999948] [G loss: 1.000100]
epoch:19 step:76495[D loss: 0.999973] [G loss: 1.000034]
epoch:19 step:76500[D loss: 0.999949] [G loss: 1.000055]
epoch:19 step:76505[D loss: 0.999966] [G loss: 1.000070]
epoch:19 step:76510[D loss: 0.999985] [G loss: 1.000051]
epoch:19 step:76515[D loss: 0.999989] [G loss: 0.999999]
epoch:19 step:76520[D loss: 0.999933] [G loss: 1.000104]
epoch:19 step:76525[D loss: 0.999993] [G loss: 1.000056]
epoch:19 step:76530[D loss: 1.000103] [G loss: 0.999819]
epoch:19 step:76535[D loss: 0.999937] [G loss: 1.000074]
epoch:19 step:76540[D loss: 0.999978] [G loss: 1.000102]
epoch:19 step:76545[D loss: 0.9

epoch:19 step:77145[D loss: 1.000129] [G loss: 0.999904]
epoch:19 step:77150[D loss: 0.999886] [G loss: 1.000136]
epoch:19 step:77155[D loss: 0.999926] [G loss: 1.000107]
epoch:19 step:77160[D loss: 0.999948] [G loss: 1.000080]
epoch:19 step:77165[D loss: 0.999981] [G loss: 1.000055]
epoch:19 step:77170[D loss: 1.000146] [G loss: 0.999824]
epoch:19 step:77175[D loss: 1.000032] [G loss: 0.999926]
epoch:19 step:77180[D loss: 0.999927] [G loss: 1.000063]
epoch:19 step:77185[D loss: 0.999863] [G loss: 1.000154]
epoch:19 step:77190[D loss: 1.000024] [G loss: 1.000009]
epoch:19 step:77195[D loss: 0.999996] [G loss: 1.000080]
epoch:19 step:77200[D loss: 1.000001] [G loss: 1.000091]
##############
[0.84303256 0.85287707 0.82976861 0.82821012 0.79252345 0.83114519
 0.8842254  0.8261698  0.82927184 0.84746013]
##########
epoch:19 step:77205[D loss: 1.000000] [G loss: 1.000007]
epoch:19 step:77210[D loss: 1.000046] [G loss: 1.000057]
epoch:19 step:77215[D loss: 1.000214] [G loss: 0.999752]
epoch:

epoch:19 step:77820[D loss: 0.999948] [G loss: 1.000033]
epoch:19 step:77825[D loss: 0.999960] [G loss: 1.000027]
epoch:19 step:77830[D loss: 0.999999] [G loss: 1.000108]
epoch:19 step:77835[D loss: 0.999972] [G loss: 1.000024]
epoch:19 step:77840[D loss: 0.999969] [G loss: 1.000080]
epoch:19 step:77845[D loss: 0.999985] [G loss: 1.000034]
epoch:19 step:77850[D loss: 0.999973] [G loss: 1.000104]
epoch:19 step:77855[D loss: 0.999994] [G loss: 1.000052]
epoch:19 step:77860[D loss: 1.000014] [G loss: 0.999940]
epoch:19 step:77865[D loss: 0.999952] [G loss: 1.000109]
epoch:19 step:77870[D loss: 0.999990] [G loss: 1.000061]
epoch:19 step:77875[D loss: 0.999902] [G loss: 1.000156]
epoch:19 step:77880[D loss: 1.000002] [G loss: 1.000079]
epoch:19 step:77885[D loss: 0.999955] [G loss: 1.000114]
epoch:19 step:77890[D loss: 0.999959] [G loss: 1.000148]
epoch:19 step:77895[D loss: 0.999983] [G loss: 1.000068]
epoch:19 step:77900[D loss: 0.999990] [G loss: 1.000029]
epoch:19 step:77905[D loss: 1.0